After this assignment was released, it seems Google has raised the minimum version of CUDA/CUDNN available on free Colab instances, which means we have to now manually downgrade both CUDA and CUDNN. We apologize for the inconvenience!

Before you connect to a runtime, please sign up for an NVIDIA developer account (which is free) so that you can download CUDNN7. Unfortunately we are not allowed to redistribute it due to licensing issues. 

Once you are signed up and logged in, use this link to download CUDNN7 locally:

[Download cuDNN v7.6.5 (November 5th, 2019) Runtime Library for Ubuntu18.04 (Deb), for CUDA 10.0](https://developer.nvidia.com/compute/machine-learning/cudnn/secure/7.6.5.32/Production/10.0_20191031/Ubuntu18_04-x64/libcudnn7_7.6.5.32-1%2Bcuda10.0_amd64.deb) (165MB)

which is found on https://developer.nvidia.com/rdp/cudnn-archive

**The above link ensures you are downloading the correct version of CUDNN7**

If you need to download it manually, then you will need to select v7.6.5 for CUDA 10.0 and then select cuDNN Runtime Library for Ubuntu18.04 (Deb), but if you are logged in, the above link should work.

Once you have downloaded it locally, you will then need to upload it to the Colab instance and install it, as instructed below. For now, once it is saved locally to your computer, proceed to the next step, which is changing the runtime type to GPU in the runtime menu. 



 **Note: when you aren't actively using the GPU for training, colab can be quick to disconnect you, so you might want to read through this notebook and familiarize yourself with the steps you'll take before you connect to a runtime/change the runtime type. Also note that if you upload data to Colab directly (and don't use Google Drive), and then change the runtime type, you will have to reupload your data. It may be easier to use Google Drive: see [here for tips on how to do so](https://colab.research.google.com/notebooks/io.ipynb).**

When you are ready to train, start by changing the runtime type to 'GPU' in the Runtime menu.

Next, we need to install Tensorflow V1 in order to run the fine-tuning code. The below cell uninstalls TF2 and installs TF1. 


In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow-gpu==1.15

Make sure you install TF version 1.15.0 using the above cell, and click the "RESTART RUNTIME" button in the output of the previous cell (this button doesn't always show up automatically) or by using the Runtime menu after the previous cell finishes executing.

In [ ]:
import tensorflow as tf
print(tf.__version__)
assert(tf.VERSION == "1.15.0") #Make sure you install Tensorflow 1.15.0 and restart the runtime!

Now it is time to downgrade CUDA to 10.0 (which is compatible with TF1.15). Run the following cell and when it successfully completes, **RESTART THE RUNTIME AGAIN**

Note that this cell can take a painfully long time to complete. In particular it might spend a few minutes on the following step (compiling a new kernel module):

`Building initial module for 4.15.0-193-generic`

While it is running you can start uploading `libcudnn7_7.6.5.32-1+cuda10.0_amd64.deb` (read the text cell two steps below this one for details)

In [ ]:
#Uninstall the current CUDA version
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

#Download CUDA 10.0
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
#install CUDA kit dpkg
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0

**After you have restarted the runtime again** check to make sure `nvcc` reports the correct version of CUDA is installed:

In [ ]:
!nvcc --version

Now upload `libcudnn7_7.6.5.32-1+cuda10.0_amd64.deb` to Colab (you can click on the 'Files' button on the left and drag it in there) and **after it has finished uploading** (which can take a long time depending on your connection speed plus how busy the Colab instance is), run the following cell to install CUDNN7 (you will **not** need to restart the runtime after installing CUDNN7 - whew!)

Thanks for your patience! Now TF 1.15 is supported again!

In [ ]:
!dpkg -i libcudnn7_7.6.5.32-1+cuda10.0_amd64.deb

You can verify `libcudnn7` is present by running the following cell (it's ok it libcudnn8 is also present, but you should see `7.6.5.32-1+cuda10.0` in the list below)

In [ ]:
!dpkg -l | grep -i cudnn

Let's install another dependency we will need for training:

In [ ]:
!pip install tf_slim

Now we'll clone the homework assignment and change into the cloned directory

In [ ]:
!git clone https://github.com/mmaz/cs249r-a2
import os
os.chdir("/content/cs249r-a2")
!pwd

Next, upload your image data into two folders under the folder `custom_dataset/TL_Datasets` - these folders are hardcoded, but you will create two folders within `TL_Datasets` with names corresponding to category labels you choose. 

You need to make sure for each category, there is a separate folder. For example, if you are downloading images of people not wearing masks, you will save these images in the `absent mask` folder (you can choose whatever name you want!) folder. Likewise, if you download images with a person wearing a mask, you will save it `mask` folder. Once you have images for each category in their respective folders, keep the parent folder named `TL_Datasets` (the conversion script expects this name!). Your folder tree in colab should resemble something like this:

  ```
├── content
│   └── cs249r-a2
│       └── custom_dataset
│           └── TL_Datasets
│               ├── absent_mask
│               └── mask

  ```

  If uploading your data to Colab is taking a long time, see the tips section in the Assignment 2 writeup on compressing your images before uploading.

**Dataset Preparation**

Once you have downloaded the dataset and a folder structure as described above, run the next cell to convert your data into a set of sharded `train` and `val` TF Records.

**Note:** you can modify a hardcoded parameter in the script (`datasets/download_and_convert_custom.py`) before running the cell to change the number of validation images saved as TF records. A decent rule of thumb is 10% or 20% of your data (so 350 implies a finetuning set of ~1750 images following this rule). 

```python
# The number of images in the validation set.
_NUM_VALIDATION = 350
```

Lastly, keep in mind how balanced your dataset is between your two classes. 

The script shuffles samples at random.

In [ ]:
!python datasets/download_and_convert_data.py \
    --dataset_name=custom \
    --dataset_dir=/content/cs249r-a2/custom_dataset/


If the script runs successfully, then you should see output similar to:

  ```
  >> Converting image 134/558 shard 1
  >> Converting image 350/350 shard 4

  Finished converting the dataset!
  ```

Use the below cell to inspect the TL_Datasets folder, where you will find train and validation TFRecord files are generated. Also, you will find a `labels.txt` file, which will have class labels. Depending upon how you organized your folders, you will find that `0` and `1` class labels will be assigned to that category (e.g., since ‘absent_mask’ is lexically lower than ‘mask’ it gets a label id of 0). This should help you interpret/modify your LED light behavior and serial output in [`arduino_detection_responder.cpp`](https://github.com/tinyMLx/arduino-library/blob/main/examples/person_detection/arduino_detection_responder.cpp)

In [ ]:
!ls custom_dataset/
!echo "" && cat custom_dataset/labels.txt

We are ready to start transfer learning! The below cell will start executing the transfer learning process on top of a pre-trained checkpoint (this checkpoint was trained on Visual Wake Words for 1 million steps). You can modify the below cell to change the frequency of saved checkpoints to your liking (it defaults to every 200 seconds), but keep an eye on the training process so that you don't leave it running for too long (colab will eventually disconnect you). When using a GPU runtime you should be able to get ~5000 steps trained within 20 minutes or so, give or take, and not face any disconnection risks.

When you think your model has been trained enough, you can click `Runtime -> Interrupt Execution` to kill training. Then you can move on to evaluation. If you find the model wasn't trained enough, you can either collect more data, train for longer, or both. If you suspect you are overfitting, either train for fewer steps (or use an earlier checkpoint) or collect more data.

By default, checkpoints will be saved to `/content/cs249r-a2/transferlearning/`

The argument `--dataset_name=visualwakewords` in this cell and cells below is due to hardcoded behavior in the training script - you can leave this as is (or you can update the training script if you prefer).

You can experiment with changing `--checkpoint_exclude_scopes`, and `--trainable_scopes` flags. For `--checkpoint_exclude_scopes`, you will list the node names (separated by comma) that you exclude when loading the pre-trained weights. `--trainable_scopes` means which layer you want to train. Obviously, this value should be the same as what you passed for the `--checkpoint_exclude_scopes` parameter since it is only for those layers you are not initializing the weights from the pre-trained model. For MobileNet, `MobilenetV1/Logits, MobilenetV1/Predictions` corresponds to the last two layers. However, if you want to play with other layers, you can use [Netron](https://netron.app/) to visualize the node names. So except for these nodes, the weights from the pre-trained model will be initialized in the model.

For the other hyperparameters, always start with the hyperparameters below that are known to work. We want you to appreciate that hyperparameters are important for making any neural network work. But if you start tweaking these (esp for computer vision tasks), two weeks (duration of this assignment) is insufficient for this endevour (unless you are extremely lucky!). So choose your battles wisely!

In [ ]:
!python train_image_classifier.py \
    --train_dir=transferlearning/  \
    --dataset_dir=/content/cs249r-a2/custom_dataset/ \
    --dataset_name=visualwakewords \
    --dataset_split_name=train \
    --preprocessing_name=mobilenet_v1 \
    --model_name=mobilenet_v1_025 \
    --train_image_size=96 \
    --save_summaries_secs=200 \
    --save_interval_secs=200 \
    --max_number_of_steps=1005000 \
    --learning_rate=0.045 \
    --label_smoothing=0.1 \
    --learning_rate_decay_factor=0.98 \
    --num_epochs_per_decay=2.5 \
    --moving_average_decay=0.9999 \
    --use_grayscale=True \
    --checkpoint_path=ckpts/person_detection/model.ckpt-1000000 \
    --checkpoint_exclude_scopes= MobilenetV1/Logits,MobilenetV1/Predictions \
    --trainable_scopes= MobilenetV1/Logits,MobilenetV1/Predictions


The next cell will evaluate your trained model on your validation set. This can be used as feedback to guide your collection of additional data and retraining the model (e.g., how many steps are ideal). 

Change the `--checkpoint` argument to point to the checkpoint you wish to evaluate in the `transferlearning/` folder.

**Remember to report the final validation accuracy you achieve in your Assignment 2 writeup**

The accuracy will be reported as (for example):

```
eval/Accuracy[0.75714287]
```

In [ ]:
!python eval_image_classifier.py \
    --alsologtostderr \
    --checkpoint_path=transferlearning/model.ckpt-1129 \
    --dataset_dir=/content/cs249r-a2/custom_dataset/ \
    --dataset_name=visualwakewords \
    --dataset_split_name=val \
    --model_name=mobilenet_v1_025 \
    --preprocessing_name=mobilenet_v1 \
    --use_grayscale=True \
    --train_image_size=96

The next steps will help you convert your model to a C++ source file so that you can load it on the Arduino Nano 33 BLE. 

First, export the model graph by running the following cell. This command will create the graph definition for the neural network model that you used (MobileNetV1). Unless you change the name of the `---output_file`, the name of the graph def will be `vww_96_grayscale_graph`. (it's fine to leave it as is).

This does not contain your model weights, just placeholders for where your weights belong (which you'll fill in using the cell after this one)

In [ ]:
!python export_inference_graph.py \
    --alsologtostderr \
    --dataset_name=visualwakewords \
    --model_name=mobilenet_v1_025 \
    --image_size=96 \
    --use_grayscale=True \
    --output_file=vww_96_grayscale_graph.pb

The next cell will freeze your weights into a new protobuf. This command will store the weights values as constants in the corresponding nodes in the graph definition you generated above. Also in this step, the script will prune the nodes that are not needed in the forward inference (e.g., nodes that are needed for gradient computation in backpropagation are all pruned from the graph). The parameters in the script are explained below:


  `--input_graph` is the name of the graph def file that you saved above.

  `--input_checkpoint` is the model name of your final trained model. 
  
  **Change `--input_checkpoint` to point to your desired checkpoint**

  `--output_graph` is the name of the frozen graph. If you don't change the parameter, by default, it will be saved as `person_detection_frozen.pb`

  `--output_node` is the name of the last node in your neural network model. We are using MobileNetv1, so the node name will be `MobilenetV1/Predictions/Reshape_1`

In [ ]:
!python freeze_graph.py \
    --input_graph=vww_96_grayscale_graph.pb \
    --input_checkpoint=transferlearning/model.ckpt-5000 \
    --input_binary=true \
    --output_graph=person_detection_frozen.pb \
    --output_node_names=MobilenetV1/Predictions/Reshape_1

Now we are ready to create a TFLite file by quantizing the model. This script will output `person_detection.tflite` (the output filename is hardcoded - as usual, you can leave it as is).

In particular, this script quantizes the weights to an int8 format. For effective quantization, you need to provide a representative input sample so the quantization script can monitor the range, which it will use it during quantization. Use the `--sample_data` flag to point to a representative input dataset. By default, it points to the first shard of your training data. The second input (`--frozen_model`) to the quantization script is the frozen model that you created in the previous step.

In [ ]:
!python quantize_weights.py --sample_data custom_dataset/train.record-00000-of-00005 --frozen_model person_detection_frozen.pb

Once you have generated the TFLite file, you will convert the TFLite file to a C++ source file to deploy it on the Arduino Nano 33 BLE using `xxd` in the following command.

If you haven't heard of `xxd` before, the following pages offer a brief explanation:

* https://en.wikipedia.org/wiki/Hex_dump
* https://manpages.ubuntu.com/manpages/bionic/en/man1/xxd.1.html



In [ ]:
!xxd -i person_detection.tflite person_detect_model_data.cpp

Finally, download `person_detection.tflite` and `person_detect_model_data.cpp` to your local computer and import it into your arduino sketch as described in the Assignment 2 writeup. 

You will need the `tflite` file in order to calculate the TensorFlow Lite Micro arena size. The `.cpp` file contains your serialied TFLM model definition and weights.